<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/Learning_Transfer_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";  # The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="4";  # Do other imports now...

# Billiotecas

In [ ]:
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
from sklearn.metrics import confusion_matrix ,accuracy_score,cohen_kappa_score,roc_curve,auc,recall_score,precision_score,f1_score
from keras.models import load_model

# Base

In [ ]:
train_path = '/content/drive/My Drive/CT-COVID-19/Train/'
test_path = '/content/drive/My Drive/CT-COVID-19/Test/'
val_path = '/content/drive/My Drive/CT-COVID-19/Val/'
num_classes = 2
#Caminho das dados e quantidade de classes
#veja que dentro da pasta Train existe uma separação por pastas contendo as classes, e isso para todas as outras.
#exemplo '../_V2-GS1/Train/Glaucoma' '../_V2-GS1/Train/Normal'

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(224,224),color_mode='rgb',batch_size=64,shuffle=True)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(224,224),color_mode='rgb',batch_size=64,shuffle=True)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path, target_size=(224,224),color_mode='rgb',batch_size=64,shuffle=True)
#aqui vc vai carregar uma referencia de todo o conjunto de dados separados por bathcs

Found 422 images belonging to 2 classes.
Found 214 images belonging to 2 classes.
Found 86 images belonging to 2 classes.


In [ ]:
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
model.layers.pop()
new_layer = Dense(num_classes, activation='softmax', name='prediction')
model = Model(model.input, new_layer(model.layers[-1].output))
#aqui é bem simples, é apenas a criação do modelo, já com os pessos carregados da imagenet

# Predição

In [ ]:
predicts_vectors = model.predict_generator(test_batches, verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
4/4 [==============================] - 67s 17s/step


In [ ]:
pred = np.argmax(predicts_vectors,axis=1)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer ()

lb.fit(test_batches.labels)

y_test = lb.transform (test_batches.labels)
y_pred = lb.transform (pred)

auc_score = roc_auc_score(y_test, y_pred,average="macro")

In [ ]:
y_true = test_batches.labels
y_pred = pred

recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
kappa = cohen_kappa_score(test_batches.labels, pred)
acc = accuracy_score(test_batches.labels, pred)

In [ ]:
print('>> Resultados <<')
print('Acc: {}'.format(acc*100))
print('Recall: {}'.format(recall*100))
print('Precision: {}'.format(precision*100))
print('F1-Acore: {}'.format(f1))
print('AUC: {}'.format(auc_score))
print('Kappa: {}'.format(kappa))

>> Resultados <<
Acc: 51.4018691588785
Recall: 51.20629370629371
Precision: 51.22950819672132
F1-Acore: 0.5105559465165377
AUC: 0.512062937062937
Kappa: 0.02420203437390378


In [ ]:
confusion_matrix(y_true,y_pred)

array([[64, 46],
       [58, 46]])